In [ ]:
# Baseline real hardware

In [14]:
# necessary libraries: Circuit Construction, transpilation, For simulation, visualization, JSON
# Circuit Construction
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

#Transpilation
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# For Simulation
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
aer_sim = AerSimulator()                              # for ideal simulation
service = QiskitRuntimeService()
backend_name = "ibm_sherbrooke"
backend = service.get_backend(backend_name)            # for real hardware


from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler_ideal = Sampler(backend=aer_sim)
sampler_noisy = Sampler(backend=backend)
options = sampler_noisy.options
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = 'XY4'
#Visualization
from qiskit.visualization import plot_histogram



#Post-processing, Fidelity and JSON
import json
import matplotlib.pyplot as plt
import statistics

from qiskit.quantum_info.analysis import hellinger_fidelity

In [15]:
# function for Circuit Construction (GHZ state for how many number of qubits, that is num_qubit argument of the function)

def get_ghz_circ_static(num_qubit):
    if num_qubit%2==1:                                                     #odd no of qubits
        qr=QuantumRegister(num_qubit)
        cr2=ClassicalRegister(num_qubit,name="cr2")                     
        qc=QuantumCircuit(qr,cr2)
        mid_qubit_index=int((num_qubit-1)/2)
        qc.h(mid_qubit_index)
        for i in range(mid_qubit_index,num_qubit-1):
            qc.cx(i,i+1)
            qc.cx(num_qubit-i-1,num_qubit-i-2)
        

    else:                                                                    #even no of qubits
        qr=QuantumRegister(num_qubit)
        cr2=ClassicalRegister(num_qubit,name="cr2")                     
        qc=QuantumCircuit(qr,cr2)
        mid_qubit_index=int((num_qubit-2)/2)
        qc.h(mid_qubit_index)
        for i in range(mid_qubit_index,num_qubit-2):
            qc.cx(i,i+1)
            qc.cx(num_qubit-1-i-1,num_qubit-i-3)
        qc.cx(num_qubit-2,num_qubit-1)
    for i in range(num_qubit):
        qc.measure(qr[i],cr2[i])
    return qc

qc=get_ghz_circ_static(6)

In [16]:
# Transpilation ideal
pm_i = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
qc_ideal = pm_i.run(qc)

# Transpilation noisy
pm_r = generate_preset_pass_manager(backend=backend, optimization_level=3)
qc_noisy = pm_r.run(qc)

In [17]:
#Run the Circuit
result_ideal = sampler_ideal.run([qc_ideal]).result()


In [6]:
# run on real hardware
job = sampler_noisy.run([qc_noisy])
print(job.job_id())

cwhq7y7mptp0008h9d60


In [18]:
#Post-processing 
job_id = 'cwhq7y7mptp0008h9d60' 
job = service.job(job_id)
result_noisy = job.result()

In [19]:
# Get Hellinger Fidelity
counts_ideal = result_ideal[0].data.cr2.get_counts()
counts_noisy = result_noisy[0]['__value__']['data'].cr2.get_counts()
Fidelity_baseline_real = hellinger_fidelity(counts_ideal, counts_noisy)

In [20]:
Fidelity_baseline_real

0.9010413144597419

In [21]:
#Store Result into JSON file
import json
with open("Fidelity_scripts_comparison.json", "r") as jsonfile:
    data1 = json.load(jsonfile)

new_data1 =  {
    "Fidelity_baseline_real" : Fidelity_baseline_real,    
    }

    

data1.update(new_data1)

with open("Fidelity_scripts_comparison.json", "w") as jsonfile:
    json.dump(data1, jsonfile)